In [6]:
import codecs
import os
import numpy as np
from collections import Counter, OrderedDict
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
import string
import re
from nltk import pos_tag
from nltk.chunk import conlltags2tree
from nltk.tree import Tree
from operator import itemgetter
from nltk import word_tokenize
from nltk.tag.stanford import StanfordNERTagger
from nltk.tag.stanford import StanfordPOSTagger
from sklearn import preprocessing

In [7]:
class DataPopulation:
    
    def GetJournosIds():
        journos_ids = list()
        journos_file = codecs.open('ireland_ids.txt', 'r')
        for line in journos_file:
            journos_ids.append(line.strip())
        journos_file.close()

    def GetJournosInfo():
        journos_info = dict()
        journos_file = codecs.open('irish_journos_info.txt')
        for line in journos_file:
            parts = line.split('\t')
            journos_info['@'+parts[3].lower()] = '@'+parts[3].lower()+'\t'+str(parts[0])+'\t'+parts[1]+'\t'+parts[2]+'\t'+parts[4]+'\t'+parts[5]+'\t'+parts[6].lower()+'\t'+parts[7].strip().lower()
        journos_file.close()

    def GetMentionedJournos():
        mentioned_journos_file = codecs.open('mentioned_journalists_ireland.txt')
        mentioned_journos_list= list()
        for line in mentioned_journos_file:
            mentioned_journos_list.append(line.strip())
        mentioned_journos_file.close()

    def LoadAudienceInfo():
        audience_file = codecs.open('audience_descriptions_ireland.txt', 'r')
        audience_description_list = list()
        for line in audience_file:
            audience_description_list.append(line.lower().strip().split('\t'))
        audience_file.close()
        audience_df = pd.DataFrame.from_records(audience_description_list, columns = ['user_id','description','verified','user_name', 'gender','occupation','hobby','salary'])

    def GetInteractionsWithJourno(journo_name):
        interactions_df = pd.DataFrame(columns=['user_id', 'tweet_text', 'day', 'month', 'hour', 'year'])
        for file1 in os.listdir('/audience_tweets_ireland/'):
            this_user_tweets = codecs.open('/audience_tweets_ireland/'+file1, 'r')
            for line in this_user_tweets:
                pieces = line.split('\t')
                while len(pieces) < 2:
                    line = line.replace('\n', ' ').replace('\r',' ') + next(this_user_tweets)
                    pieces = line.split('\t')
                tweet_text = pieces[0]
                tweet_timestamp = pieces[1]
                if journo_name in tweet_text:
                    interaction = list()
                    interaction.append(file1.replace('.txt',''))
                    interaction.append(tweet_text.replace('&amp;', '&'))
                    timestamp = tweet_timestamp.strip().split(' ')
                    interaction.append(timestamp[0])
                    interaction.append(timestamp[1])
                    interaction.append(timestamp[3])
                    interaction.append(timestamp[5])
                    interaction_df = pd.DataFrame(np.array(interaction).reshape(1,6), columns=['user_id', 'tweet_text', 'day', 'month', 'hour', 'year'])
                    interactions_df.index = range(1,len(interactions_df)+1)
                    interactions_df = interactions_df.append(interaction_df)
            this_user_tweets.close()
                
    def GetCoocurrences(mentioned_journos_list):
        user_journos = dict()
        #create empty dataframe with headings
        all_journos_mentioned = pd.DataFrame(columns=mentioned_journos_list)
        #extract users interactions, create a dataframe for each user and append it to a global dataframe
        for file1 in os.listdir('audience_tweets_ireland/'):
            if file1.replace('.txt','') not in journos_ids:
                user_journos = user_journos.fromkeys(mentioned_journos_list, 0)
                cat_days_blocks = Counter()
                this_user_tweets = codecs.open('audience_tweets_ireland/'+file1, 'r')
                for line in this_user_tweets:
                    pieces = line.split('\t')
                    while len(pieces) < 2:
                        line = line.replace('\n', ' ').replace('\r',' ') + next(this_user_tweets)
                        pieces = line.split('\t')
                    tweet_text = pieces[0]
                    for k in journos_info.keys():
                        if k in tweet_text:
                            this_journo = k+'\t'+journos_info[k]
                    if this_journo != '':
                        journo_bits = this_journo.split('\t')
                        user_journos[journo_bits[1]] = 1
                result_df = pd.DataFrame(user_journos, index=[file1.replace('.txt','')])
                all_journos_mentioned = all_journos_mentioned.append(result_df)
                this_user_tweets.close()
        df_asint = all_journos_mentioned.astype(int)
        coocc = df_asint.T.dot(df_asint)
        df_this_journo_cooc = pd.DataFrame(coocc[this_journalist].reset_index(name='total_interactions'))
        df_this_journo_cooc = df_this_journo_cooc.loc[df_this_journo_cooc['total_interactions'] > 4]
        df_this_journo_cooc = df_this_journo_cooc.rename(columns={'index': 'journalist'})
    

In [8]:
DataPopulation.GetJournosIds()
DataPopulation.GetJournosInfo()
DataPopulation.LoadAudienceInfo()
DataPopulation.GetInteractionsWithJourno(this_journalist)
DataPopulation.GetCoocurrences(DataPopulation.GetMentionedJournos())

FileNotFoundError: [Errno 2] No such file or directory: 'ireland_ids.txt'